<a href="https://colab.research.google.com/github/lucianosilva-github/visaocomputacional/blob/master/Aula%2019/Visa%CC%83o_Computacional_Aula_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **VISÃO COMPUTACIONAL - AULA 19**

**Objetivos da aula:**

*   revisar os conceitos de Visão Estéreo, Geometria Epipolar, Matriz Fundamental e Matriz Essencial
*   introduzir os conceitos de geometria de múltiplas visões, geometria trifocal e tensor trifocal
*   praticar com o módulo pymvg


### <b> REVISÃO DE GEOMETRIA EPIPOLAR </b>

Quando trabalhamos com o modelo de câmera pinhole, perdemos uma noção importante: profundidade. Uma das maneiras para se perceber profundidade é com o uso de câmeras, conhecida como <b> Visão Estéreo</b>. 

<img src="https://opencv24-python-tutorials.readthedocs.io/en/latest/_images/epipolar.jpg"> </img>

Usando somente a câmera de centro <i>O</i>, temos várias possibilidades de pontos X que podem ser projetados no ponto x da imagem. Porém, se consideramos uma outra câmera <i>O'</i>, os pontos x' correspondentes aos pontos x levam a um único ponto X possível no espaço, que pode ser calculado através da intersecção das semiretas <i>Ox</i> e <i>O'x'</i>. 

As linhas <i>l</i> e <i>l'</i> são chamadas <b>linhas epipolares</b> e contém pontos correspondentes em projeção nas duas imagens, conhecidos como <b>epipolos</b>. O plano <i>OXO'</i> é chamado de <b>plano epipolar</b>.

Conhecidos os centros <i>O</i>, <i>O'</i> e alguns pares de epipolos <i>(e,e')</i> (normalmente, dependemos da técnica utilizada, usamos oito epipolos ou mais), é possível reconstruir as matrizes de projeção das câmeras <i>O</i> e <i>O'</i>. Esta determinação de matrizes é realizada com o auxílio da <b>Geometria Epipolar</b>.

Para simplificar o nosso modelo de duas câmeras, vamos considerar duas câmeras com os mesmos parâmetros intrínsecos (distância focal, tamaho do plano de projeção, dentre outros) e posicionadas diferentemente utilizando uma translação e uma rotação.

<img src="https://opencv24-python-tutorials.readthedocs.io/en/latest/_images/essential_matrix.jpg"> </img>

Vamos designar as matrizes das câmeras de <i>K</i> e <i>K'</i>, associadas aos centros <i>O</i> e <i>O'</i>, respectivamente. Para determinar estas duas matrizes, vamos duas outras matrizes: <b> Matriz Essencial (E)</b> e a <b> Matriz Fundamental(F) </b>.

A matriz essencial E contém informações sobre a translação T e a rotação R da câmera <i>O'</i>, em relação à câmera <i>O</i>. A matriz fundamental F permite projetar uma linha epipolar de imagem na linha epipolar correspodnente na outra imagem.

### <b> MATRIZ FUNDAMENTAL F </b>

A matriz fundamental F pode ser obtida resolvendo o sistema de equações lineares abaixo:

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/6ab25460ff5b12e37a92d4ad41487802476c69be"> </img>
</center>

A matriz F é uma matrix 3x3. Assim, 8 equações são suficientes para determiná-la completamente. Estas equações são obtidas através de 8 pares de epipolos <i>(e,e')</i> correspondentes nas duas imagens.





### <b> MATRIZ FUNDAMENTAL F </b>

A matriz fundamental F pode ser obtida resolvendo o sistema de equações lineares abaixo:

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/6ab25460ff5b12e37a92d4ad41487802476c69be"> </img>
</center>

A matriz F é uma matrix 3x3. Assim, 8 equações são suficientes para determiná-la completamente. Estas equações são obtidas através de 8 pares de epipolos <i>(e,e')</i> correspondentes nas duas imagens.

### <b> MATRIZ ESSENCIAL E </b>

A matriz fundamental E satisfaz a equação abaixo, onde K e K' são as matrizes das duas câmeras e, F, a matriz fundamental.


Se soubermos a matriz essencial E, podemos obter as matrizes associadas às câmeras. Uma maneira de fazer isto é supor que K seja a matriz identidade e, em função dela,determinar a matriz K'.





### <b> MATRIZ ESSENCIAL E </b>

A matriz fundamental E satisfaz a equação abaixo, onde K e K' são as matrizes das duas câmeras e, F, a matriz fundamental.

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/94797b318a9e805f6515cee74fa3de42ac35ee7a"> </img>
</center>

Se soubermos a matriz essencial E, podemos obter as matrizes associadas às câmeras. Uma maneira de fazer isto é supor que K seja a matriz identidade e, em função dela,determinar a matriz K'.

Obtendo a matriz E, podemos decompô-la em fatores usando a decomposição SVD (Decomposição em Valores Singulares). A decomposição SVD permite decompor uma matriz M em três componentes:

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/f577effb716a33019b63f9072fd245bb2a751bdb"> </img>
</center>

Por exemplo, a matriz M mostrada abaixo

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/0d5d3df9cf619062481ab2a6b328467990152f1a"> </img>
</center>

possui a seguinte decomposição em valores singulares

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/73fdf5fe339a08566cccb833c860558092532f74"> </img>
</center>

A matriz do centro é sempre uma matriz diagonal, formada por autovalores. As matrizes U e V* são matrizes unitárias, isto é, multiplicado elas por elas mesmas, produzimos a matriz identidade:

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/a7f128976e087ec8136d54eeaab90a2eda2a09bc"> </img>
</center>
<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/43daa2de3e7835aba9c75c1eef02bbe5c2d6f687"> </img>
</center>


Para obter uma possível matriz de rotação de uma câmera e sua translação em relação a outra câmera, podemos usar as seguintes relações:

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/982105be58f296a050d9bd1be02ec12d1c578c21"> </img>
</center>

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/bc1799dbc7da928a0d4576f5b9ed5d21d955be72"> </img>
</center>

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/563ddbeb35c985502bc0bb0b0cc13178a2b68c43"> </img>
</center>

<center>
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/f48a92bc850bdfa4b0e313281f1ba9ae8a0af0ac"> </img>
</center>

#### <b> GEOMETRIA COM TRÊS CÂMERAS (TRIFOCAL) E O TENSOR TRIFOCAL </b>

Assim como num sistema com duas imagens, podemos extrair informações sobre o cenário em 3D através da correspondência entre pontos, retas, tangentes e segmentos de curvas com três ou mais imagens. Isto é chamado genericamente de **Geometria de Múltiplas Visões**.

<center> <img src="https://miro.medium.com/max/810/1*wzzIPX7GiBOU-ON6kDp0Dg.jpeg"> </img> </center>

Nesta aula, trabalhemos com três visões, através de um sistema de três câmeras **(Geometria Trifocal)**. Nesta configuração três câmeras, temos a seguinte configuração:

<center> <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F27575648586264FE04"> </img> </center>

Neste tipo de geometria, temos três linhas epipolares: l, l' e l''. A partir delas, definimos o **tensor trifocal**: 

<center> <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F24147A4B5862653424"> </img></center>

As matrizes de câmeras são calculadas a partir do tensor trifocal. Para calcular o tensor trifocal, precisamos de **seis pontos em cada imagem**   correspondentes nas três imagens. Observe que esta quantidade é menor que na **visão bifocal (visão estéreo)**, onde precisávamos de **8 pontos**. 





